## Final Model 


In [5]:
# TensorFlow and tf.keras
import tensorflow as tf
# recommended models
from keras.applications import vgg16
from keras.applications.vgg19 import VGG19
from keras.applications import ResNet50
from tensorflow.keras.utils import load_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import Sequential, model_from_json
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
import keras

#CNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

import datetime
# from tensorflow.keras.callbacks import TensorBoard
from keras.utils import to_categorical
#import resnet50
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
# get the array with the class names from the folder 
# read foldernames
import os
import glob
import cv2



In [6]:
# read all the paths from the folder
def get_paths(folder):
    paths = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            # if file.endswith(".jpg"):
            paths.append(os.path.join(root, file))
    return paths

In [7]:
shop_images = get_paths(
    r'C:\Users\mrpal\OneDrive\Desktop\WBS\fashcam\asos')


## function that loads all models

In [4]:
from keras.applications.vgg19 import VGG19


def get_models():
  # load the vgg models
  vgg16_model = vgg16.VGG16(weights='imagenet')
  vgg19_model = VGG19(weights='imagenet', include_top=False)
  # remove the last layers in order to get features instead of predictions
  feat_extractor_vgg19 = Model(inputs=vgg19_model.input,
                               outputs=vgg19_model.get_layer("block5_pool").output)
  feat_extractor = Model(inputs=vgg16_model.input,
                         outputs=vgg16_model.get_layer("fc2").output)
  # get the model with the last layer in order to get predictions
  classifier = Model(inputs=vgg16_model.input,
                     outputs=vgg16_model.get_layer("predictions").output)
  # print the layers of the CNN
  feat_extractor.summary()
  # feat_extractor_vgg19.summary()
  classifier.summary()
  return feat_extractor, classifier, feat_extractor_vgg19


In [ ]:
feat_extractor = get_models()[0]
classifier = get_models()[1]
feat_extractor_vgg19 = get_models()[2]


### function that uses resnet to predict the class and the sparse matrix of image given

 ResNet-50 is 50 layers deep and is trained on a million images of 1000 categories from the ImageNet database.
 the model has over 23 million trainable parameters

In [8]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2


def get_prediction_resnet(img_path):

    # Load the pre-trained ResNet50 model, with the top layer removed
    model = InceptionResNetV2(weights='imagenet', include_top=True)

    # Load an image to use for prediction
    img = load_img(img_path, target_size=(299, 299))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Get the predictions from the model
    features = model.predict(x)

    # Print the top 5 predictions
    predictions = decode_predictions(features, top=2)
    # for p in predictions[0]:
    #     print(f"Class: {p[1]}, Probability: {p[2]:.2f}")
    propability = predictions[0][0][2]
    return predictions[0][0], propability


In [3]:
image_names = os.listdir('asos/only')


## here we drop outliers or misclasifications and define our unique types

In [ ]:
# read the folder names from the C:\Users\mrpal\OneDrive\Desktop\WBS\fascam\clothing-dataset-small\train save them to a list
unique_types = [
    folder for folder in os.listdir(r'C:\Users\mrpal\OneDrive\Desktop\fashcam-v3\finalDataset')]


unique_types


SNIPPET TO REMOVE EVERY PHOTO FROM THE FOLDERPATH TO THE OTHER FOLDERS THAT BELONG

In [8]:

#import shutil
#folderpath= r'C:\Users\mrpal\OneDrive\Desktop\WBS\machineLearning\supervised\deeplearning\Imagesfromfashiondataset\only' 
# # for each row in the dataframe
# for index, row in selected_styles.iterrows():
#     # get the image name
#     image_name = row['image_name']
#     # get the type of clothing
#     type = row['articleType']
#     # get the source path to the image
#     src = os.path.join(r"folderpath", image_name).replace("\\", "/")
  
#     # get the destination path to the image
#     dst = r'Imagesfromfashiondataset/'+os.path.join(type, image_name).replace("\\", "/")
#     # move the image from the source to the destination
#     # print(src, dst)
#     shutil.move(src, dst)


In [16]:
# give the number of files in the folder with the given path
def get_number_of_files(path):
    return len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))])
for i in range(len(unique_types)):
    print(unique_types[i], get_number_of_files(r'C:\Users\mrpal\OneDrive\Desktop\fashcam-v3\finalDataset' + '/' + unique_types[i]))    

Backpacks 200
Belts 611
Bra 477
Caps-hats 180
CasualShoes 572
Dresses 659
Earrings 337
Handbags 94
Heels 43
Leggings 177
Outwear 299
pijamas 141
Ring 118
Sandals 361
Scarves 119
Shirts 459
Shorts 327
Skirts 152
Sportswear 304
Sunglasses 1073
Sweatshirts 582
Tops 257
Trousers 896
Tshirts 3072


In [17]:

train_ds, test_ds = tf.keras.utils.image_dataset_from_directory(
    r"C:\Users\mrpal\OneDrive\Desktop\fashcam-v3\finalDataset",
    labels="inferred",
    validation_split=0.4,
    shuffle=True,
    subset="both",
    seed=1337,
    batch_size=32,
    image_size=(224, 224),
    label_mode='categorical',
    color_mode='rgb',
    
            )


Found 11510 files belonging to 24 classes.
Using 6906 files for training.
Using 4604 files for validation.


In [52]:
# data = tf.keras.utils.image_dataset_from_directory(

#     r"C:\Users\mrpal\OneDrive\Desktop\fashcam-v3\finalDataset",
#     labels="inferred",
#     label_mode='categorical',
#     class_names=None,
#     color_mode="rgb",
#     batch_size=32,
#     image_size=(224, 224),
#     shuffle=True,
#     seed=1337,
#     validation_split=0.4,
#     subset="both",
#     interpolation="bilinear",
#     follow_links=False,
#     crop_to_aspect_ratio=False,
# )
# split the data into train and test
# train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     r"C:\Users\mrpal\OneDrive\Desktop\fashcam-v3\finalDataset",
#     validation_split=0.2,
#     subset="training",
#     seed=1337,
#     label_mode='categorical',
#     class_names=None,
#     color_mode="rgb",
#     batch_size=32,
#     image_size=(224, 224),
#     image_size=image_size,
# )
# val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     r"C:\Users\mrpal\OneDrive\Desktop\fashcam-v3\finalDataset",
#     validation_split=0.2,
#     subset="validation",
#     seed=1337,
#     label_mode='categorical',
#     class_names=None,
#     color_mode="rgb",
#     batch_size=32,
#     image_size=(224, 224),
# )


In [18]:
# import output 
def train_model_tuning():
    # Load the pre-trained ResNet50 model, with the top layer removed
    resnet50_model = InceptionResNetV2(
        weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    #model with 1 layer
    cnn_model = resnet50_model.output
    cnn_model = Flatten()(cnn_model)
    cnn_model = Dense(len(unique_types), activation='softmax')(cnn_model)
    # cnn_model = Dense(len(unique_types), activation='softmax')(cnn_model)
    final_model = Model(inputs=resnet50_model.input, outputs=cnn_model)

    # Freeze all layers in the base model (except for the top layer)
    for layer in resnet50_model.layers:
        layer.trainable = False
    # Compile the model
    final_model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

    # Convert the target labels into one-hot encoded format
    checkpoint_filepath = r'C:\Users\mrpal\OneDrive\Desktop\fashcam-v3'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
    
    # Train the model
    
    history = final_model.fit(
        train_ds,
        batch_size=100, 
        epochs=1, 
        verbose=1,
        callbacks=[model_checkpoint_callback],
        )
  
    # save the model
    final_model.save("final_model_v3.h5")
    # serialize model to JSON
    # model_json = final_model.to_json()
    # with open("final_model.json", "w") as json_file:
    #     json_file.write(model_json)
    # # serialize weights to HDF5
    # final_model.save_weights("final_model.h5")
    print("Saved model to disk")
    
    return 0

In [ ]:
# import load model
from keras.models import load_model

loaded_model = load_model('final_model_v3.h5')

# evaluate loaded model on test data
loaded_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])       
score = loaded_model.evaluate(test_ds[:100], verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100)) 

In [31]:


from keras.models import load_model
loaded_model = load_model('final_model_v3.h5')
loaded_model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model.predict(input_data)


TypeError: 'BatchDataset' object is not subscriptable

In [23]:
# confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns   
import matplotlib.pyplot as plt 
loaded_model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

y_pred = loaded_model.predict(test_ds)
y_pred = np.argmax(y_pred, axis=1)
y_true = test_ds.classes
cm = confusion_matrix(y_true, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt='.2f', xticklabels=unique_types, yticklabels=unique_types)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

### function that uses fine tunin with resnet50 to predict the class and the sparse matrix of image given

In [15]:
def get_prediction_tuning(img_path):
    # load json and create model
    json_file = open('final_model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("final_model.h5")
    # Load an image to use for prediction
    img = load_img(img_path, target_size=(224, 224,3))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    # result = loaded_model.predict(x)
    # evaluate loaded model on test data
    loaded_model.compile(optimizer='adam', loss='categorical_crossentropy',
                         metrics=['accuracy'])
    result = loaded_model.predict(x)
    # get the percentage of the prediction
    
    
    return (result,unique_types[np.argmax(result)])
    
    
    
  


In [21]:
# # confusion matrix
#import seaborn as sns
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
Y_pred = loaded_model.predict(test_x[:50])
y_pred = np.argmax(test_y, axis=1)
y_true = np.argmax(test_y[:50], axis=1)
confusion_matrix(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)   
plt.figure(figsize=(20,20))
sns.heatmap(cm, annot=True, fmt="d")
plt.title("Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

NameError: name 'test_x' is not defined

### function that uses vgg16 model to predict the class and the sparse matrix of image given

VGG-19 is an improved version of vgg16 and it is 19 layers deep.  a pretrained version of the network trained on more than a million images from the ImageNet database. The pretrained network can classify images into 1000 object categories, such as keyboard, mouse, pencil, and many animals

In [ ]:
def get_prediction_vgg16(img_path):
    # imgs_model_width, imgs_model_height = 224, 224
    # load the model
    vgg_model = vgg16.VGG16(weights='imagenet')

    original = load_img(img_path, target_size=(224, 224))
    numpy_image = img_to_array(original)
    image_batch = np.expand_dims(numpy_image, axis=0)
    images = np.vstack([image_batch])

    processed_img = preprocess_input(images.copy())
    prediction = classifier.predict(images)
    result = feat_extractor.predict(images)
    # help the model to predict the class
    return result, vgg16.decode_predictions(prediction, top=1)[0][0][1]


In [ ]:
# # load the model
# vgg_model = vgg16.VGG16(weights='imagenet')

# # remove the last layers in order to get features instead of predictions

# feat_extractor = Model(inputs=vgg_model.input,
#                        outputs=vgg_model.get_layer("fc2").output)
# # get the model with the last layer in order to get predictions
# classifier = Model(inputs=vgg_model.input,
#                      outputs=vgg_model.get_layer("predictions").output)

# # print the layers of the CNN
# feat_extractor.summary()
# classifier.summary()

### testing all models

In [ ]:
test_file = r"C:\Users\mrpal\OneDrive\Desktop\WBS\machineLearning\supervised\deeplearning\image3.jpg"

get_prediction_vgg16(test_file)[1]
get_prediction_resnet(test_file)[1]
get_prediction_tuning(test_file)[1]
# give a black frame to the picture through load image and then show it:
original = load_img(test_file)
plt.imshow(original)
print("vgg16 says " + get_prediction_vgg16(test_file)[1])
print("resnet says " + get_prediction_resnet(test_file)[1])
print ("tuned model says " + get_prediction_tuning(test_file)[1])


1/1 [==============================] - 0s 82ms/step


: 

In [ ]:
# feat_extractor.predict(preprocess_input(np.expand_dims(load_img(r'C:\Users\mrpal\OneDrive\Desktop\WBS\machineLearning\supervised\deeplearning\FasCam\clothing-dataset-small\dress\009b3c31-fb62-45c0-be9a-37a5c238cb88.jpg', target_size=(imgs_model_width, imgs_model_height)), axis=0)))

### function that gets a list of images and returns the features of those images

In [ ]:
# function that gets a list of images and returns the features of those images
def get_features(img_paths, category_folder_name):
    imgs_model_width, imgs_model_height = 224, 224
    nb_closest_images = 5
    importedImages = []
    for f in img_paths:
        filename = f
        original = load_img(filename, target_size=(224, 224))
        numpy_image = img_to_array(original)
        image_batch = np.expand_dims(numpy_image, axis=0)

        importedImages.append(image_batch)
    images = np.vstack(importedImages)
    processed_imgs = preprocess_input(images.copy())
    # load the model
    vgg_model = vgg16.VGG16(weights='imagenet')
    # remove the last layers in order to get features instead of predictions
    feat_extractor = Model(inputs=vgg_model.input,
                           outputs=vgg_model.get_layer("fc2").output)
    # print the layers of the CNN
    feat_extractor.summary()
    imgs_features = feat_extractor.predict(processed_imgs)
    print("features successfully extracted!")
    # save it as csv file
    np.savetxt(f'{category_folder_name}.csv', imgs_features, delimiter=",")
    return imgs_features

In [ ]:
get_features(list_directory[:5000], 'imagesfromfashiondata')


### function that takes an image as input and applies the cosine similarity with the features from csv to get the closest images


In [2]:
# function that takes an image as input and applies the cosine similarity with the features from csv to get the closest images
def get_closest_images(img_path, nb_closest_images=4):
    # prediction about the class of the image
    available_class = get_prediction_tuning(img_path)
    available_class = "Outwear"
    # load the features from csv

    features = np.genfromtxt('Outwear.csv', delimiter=',')
    # get the features of the image

    model = ResNet50(weights='imagenet', include_top=True)
    feat_extractor = Model(
        inputs=model.input, outputs=model.get_layer("avg_pool").output)
    feat_extractor.summary()

    img_features = feat_extractor.predict(preprocess_input(
        np.expand_dims(load_img(img_path, target_size=(224, 224)), axis=0)))
    # get the cosine similarity
    cosSimilarities = cosine_similarity(img_features, features)

    print("-----------------------------------------------------------------------")
    print("original product:")
    original = load_img(img_path, target_size=(
        224, 224))
    plt.imshow(original)
    plt.show()
    print("-----------------------------------------------------------------------")
    print("most similar products:")
    # get the indexes of the closest images
    closest_imgs_indexes = cosSimilarities.argsort()[0][-nb_closest_images:]
    # similarity score of the closest images
    closest_imgs_similarities = cosSimilarities[0][closest_imgs_indexes]

    # get the closest images
    closest_imgs = [shopfiles[i]
                    for i in closest_imgs_indexes if i < len(shopfiles)]
    for i in range(0, nb_closest_images):

        original = load_img(closest_imgs[i], target_size=(
            224, 224))
        plt.imshow(original)
        plt.show()
        print("similarity score:", closest_imgs_similarities[i])
    return 0
